In [1]:
%cd ../..

# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# other1 = Car on intersection,
#             facing Range(50, 135) deg relative to ego.heading

# other2 = Car on intersection,
#             facing -1 * Range(50, 135) deg relative to ego.heading

# require abs(relative heading of other1 from other2) > 100 deg
# require (distance from ego to intersectionRegion) < 10

/home/eecs/chanwutk/Documents/apperception


In [2]:

import time
from os import environ
from apperception.database import database
from apperception.world import empty_world

In [3]:
if environ["AP_PORT"] is None:
    environ["AP_PORT"] = str(input('port'))

In [4]:
# with (
#   select *
#   from item_traj
#   where item_traj.object_type = 'car'
# ) as cars

# select *
# from cars as t1
# join cars as t2 on t1.cameraId = t2.cameraId
# join Cameras as cam on t1.cameraId = Cameras.id
# where
#   facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) >= -15 AND
#   facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) <= 15 AND
#   DISTANCE(cam.egoTranslation, t1.centroid, cam.timestamp) < 50 AND
#   viewAngle(t1.traj, cam.egoHeading, cam.ego_Translation, cam.timestamp) < 135 AND
#   DISTANCE(cam.egoTranslation, t2.centroid, cam.timestamp) < 50 AND
#   viewAngle(t2.traj, cam.egoHeading, cam.ego_Translation, cam.timestamp) < 135 AND
#   contained(t1.traj, road_segment_type("intersection"), cam.timestamp) and
#   contained(t2.traj, road_segment_type("intersection"), cam.timestamp) and
#   (facingRelative(t1.heading, cam.egoHeading, cam.timestamp) < -50 ANd
#   facingRelative(t1.heading, cam.egoHeading, cam.timestamp) > -135) AND
#   (facingRelative(t2.heading, cam.egoHeading, cam.timestamp) < -50 AND
#   facingRelative(t2.heading, cam.egoHeading, cam.timestamp) > -135) AND
#   DISTANCE(cam.egoTranslation, roadSection("intersection"), cam.timestamp) < 10 AND
#   (facingRelative(t1.heading, t2.heading, cam.timestamp) > 100 OR
#   facingRelative(t1.heading, t2.heading, cam.timestmap) < -100)

In [5]:
database.sql("select * from intersection limit 1;")

,id,road
0,042cf95f-6298-48f9-8955-c6df65fa4f65_inter,042cf95f-6298-48f9-8955-c6df65fa4f65


In [6]:
name = 'ScenicWorld' # world name
world = empty_world(name=name)

### Query ###
from apperception.utils import F

world = world.filter("lambda obj1, obj2, cam: " +
    "(F.like(cam.cameraId, 'scene-0000') or F.like(cam.cameraId, 'scene-0001') or F.like(cam.cameraId, 'scene-0002') or F.like(cam.cameraId, 'scene-0003') or F.like(cam.cameraId, 'scene-0004')) and "
    "obj1.object_id != obj2.object_id and " +
    "F.like(obj1.object_type, 'vehicle%') and " +
    "F.like(obj2.object_type, 'vehicle%') and " +
    "F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.ego)), -15, 15) and " +
    "F.distance(cam.ego, obj1, cam.timestamp) < 50 and " +
    "F.view_angle(obj1, cam.ego, cam.timestamp) < 70 / 2.0 and " +
    "F.distance(cam.ego, obj2, cam.timestamp) < 50 and " +
    "F.view_angle(obj2, cam.ego, cam.timestamp) < 70 / 2.0 and " +
    "F.contains_all('intersection', [obj1.traj, obj2.traj]@cam.timestamp) and "
    "F.angle_between(F.facing_relative(obj1, cam.ego, cam.timestamp), 50, 135) and " +
    "F.angle_between(F.facing_relative(obj2, cam.ego, cam.timestamp), -135, -50) and " +
    "F.minDistance(cam.egoTranslation, F.road_segment('intersection')) < 10 and " +
    "F.angle_between(F.facing_relative(obj1, obj2, cam.timestamp), 100, -100) and " +
    # "F.like(cam.filename, '%n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656773912404.jpg') and " +
    "1 == 1"
)

start = time.time()

id_time_camId_filename = world.get_id_time_camId_filename(2)

end = time.time()
print(format(end-start))

execute: filter
execute: get_id_time_camId_filename
get_id_time_camId_filename 
        SELECT table_0.itemId,table_1.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as table_0
        JOIN (SELECT * FROM item_general_trajectory) as table_1 USING (cameraId)
        JOIN Cameras USING (cameraId)
        WHERE (((Cameras.cameraId LIKE 'scene-0000') OR (Cameras.cameraId LIKE 'scene-0001') OR (Cameras.cameraId LIKE 'scene-0002') OR (Cameras.cameraId LIKE 'scene-0003') OR (Cameras.cameraId LIKE 'scene-0004')) AND (table_0.itemId<>table_1.itemId) AND (table_0.objectType LIKE 'vehicle%') AND (table_1.objectType LIKE 'vehicle%') AND angleBetween(facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.egoHeading)), (-15), 15) AND (distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp)<50) AND (viewAngle(table_0.trajCentroids, Cameras.egoHeading, Cameras.egoTranslation, Cameras.timesta

In [15]:
start = time.time()
res = database.sql("""
SELECT table_0.itemId,table_1.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as table_0
        JOIN (SELECT * FROM item_general_trajectory) as table_1 USING (cameraId)
        JOIN Cameras USING (cameraId)
        WHERE (((Cameras.cameraId LIKE 'scene-0000') OR (Cameras.cameraId LIKE 'scene-0001') OR (Cameras.cameraId LIKE 'scene-0002') OR (Cameras.cameraId LIKE 'scene-0003') OR (Cameras.cameraId LIKE 'scene-0004')) AND (table_0.itemId<>table_1.itemId) AND (table_0.objectType LIKE 'vehicle%') AND (table_1.objectType LIKE 'vehicle%') AND angleBetween(facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.egoHeading)), (-15), 15) AND (distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp)<50) AND (viewAngle(table_0.trajCentroids, Cameras.egoHeading, Cameras.egoTranslation, Cameras.timestamp)<(70/2.0)) AND (distance(Cameras.egoTranslation, table_1.trajCentroids, Cameras.timestamp)<50) AND (viewAngle(table_1.trajCentroids, Cameras.egoHeading, Cameras.egoTranslation, Cameras.timestamp)<(70/2.0)) AND (EXISTS(
        SELECT intersection.id
        FROM intersection
            JOIN SegmentPolygon
                ON SegmentPolygon.elementId = intersection.id
            JOIN unnest(ARRAY[valueAtTimestamp(table_0.trajCentroids,Cameras.timestamp),valueAtTimestamp(table_1.trajCentroids,Cameras.timestamp)]) point
                ON ST_Covers(SegmentPolygon.elementPolygon, point)
        GROUP BY intersection.id
        HAVING COUNT(point) = 2
    )) AND angleBetween(facingRelative(table_0.itemHeadings, Cameras.egoHeading, Cameras.timestamp), 50, 135) AND angleBetween(facingRelative(table_1.itemHeadings, Cameras.egoHeading, Cameras.timestamp), (-135), (-50)) AND (minDistance(Cameras.egoTranslation, roadSegment('intersection'))<10) AND angleBetween(facingRelative(table_0.itemHeadings, table_1.itemHeadings, Cameras.timestamp), 100, (-100)) AND (1=1))
""")
end = time.time()
res

,itemid,itemid,timestamp,cameraid,filename
0,33ae6abe79c54c169264744932ebb551,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:47.397423+00:00,scene-0004,samples/CAM_BACK_LEFT/n015-2018-07-18-11-18-34...
1,33ae6abe79c54c169264744932ebb551,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:47.398588+00:00,scene-0004,samples/LIDAR_TOP/n015-2018-07-18-11-18-34+080...
2,33ae6abe79c54c169264744932ebb551,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:47.404844+00:00,scene-0004,sweeps/CAM_FRONT_LEFT/n015-2018-07-18-11-18-34...
3,33ae6abe79c54c169264744932ebb551,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:47.405673+00:00,scene-0004,samples/RADAR_FRONT_RIGHT/n015-2018-07-18-11-1...
4,33ae6abe79c54c169264744932ebb551,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:47.412466+00:00,scene-0004,sweeps/CAM_FRONT/n015-2018-07-18-11-18-34+0800...
...,...,...,...,...,...
240,f54238bdd48443fcb4a019271dbca2f8,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:50.485202+00:00,scene-0004,sweeps/RADAR_FRONT_LEFT/n015-2018-07-18-11-18-...
241,f54238bdd48443fcb4a019271dbca2f8,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:50.490943+00:00,scene-0004,sweeps/RADAR_BACK_RIGHT/n015-2018-07-18-11-18-...
242,f54238bdd48443fcb4a019271dbca2f8,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:50.500140+00:00,scene-0004,sweeps/LIDAR_TOP/n015-2018-07-18-11-18-34+0800...
243,f54238bdd48443fcb4a019271dbca2f8,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:50.504844+00:00,scene-0004,sweeps/CAM_FRONT_LEFT/n015-2018-07-18-11-18-34...


In [16]:
end - start

80.01678609848022

In [17]:
start = time.time()
res = database.sql("""
WITH car as (
SELECT *
FROM item_general_trajectory
  JOIN Cameras USING (cameraId)
WHERE
  objectType LIKE 'vehicle%'
  AND (cameraID = 'scene-0000' OR cameraID = 'scene-0001' OR cameraID = 'scene-0002' OR cameraID = 'scene-0003' OR cameraID = 'scene-0004')
    AND (
      distance(
        Cameras.egoTranslation, trajCentroids, 
        Cameras.timestamp
      )< 50
    ) 
    AND (
      viewAngle(
        trajCentroids, Cameras.egoHeading, 
        Cameras.egoTranslation, Cameras.timestamp
      )<(70 / 2.0)
    ) AND
    angleBetween(
      facingRelative(
        Cameras.egoHeading, 
        roadDirection(
          Cameras.egoTranslation, Cameras.egoHeading
        )
      ), 
      (-15), 
      15
    ) 
    AND (
      minDistance(
        Cameras.egoTranslation, 
        roadSegment('intersection')
      )< 10
    ) 
), inters AS (
    SELECT elementPolygon, id
    FROM Intersection
    JOIN SegmentPolygon ON elementId = id
), car1 AS (
    SELECT *
    FROM car
    WHERE
    angleBetween(
      facingRelative(
        itemHeadings, egoHeading, 
        timestamp
      ), 
      50, 
      135
    ) 
), car2 AS (
    SELECT *
    FROM car
    WHERE
    angleBetween(
      facingRelative(
        itemHeadings, egoHeading, 
        timestamp
      ), 
      (-135), 
      (-50)
    ) 
)

SELECT 
  table_0.itemId, 
  table_1.itemId, 
  table_0.timestamp, 
  table_0.cameraId, 
  table_0.filename 
FROM 
  car1 as table_0
  JOIN car2 as table_1 USING (cameraId, timestamp, filename) 
WHERE 
  (
    table_0.itemId <> table_1.itemId AND
    (
      EXISTS(
        SELECT 
          inters.id
        FROM 
          inters
          JOIN unnest(
            ARRAY[valueAtTimestamp(
              table_0.trajCentroids, table_0.timestamp
            ), 
            valueAtTimestamp(
              table_1.trajCentroids, table_0.timestamp
            ) ]
          ) point ON ST_Covers(
            inters.elementPolygon, point
          ) 
        GROUP BY 
          inters.id 
        HAVING 
          COUNT(point) = 2
      )
    ) 
    AND angleBetween(
      facingRelative(
        table_0.itemHeadings, table_1.itemHeadings, 
        table_0.timestamp
      ), 
      100, 
      (-100)
    )
  )
""")
end = time.time()
res

,itemid,itemid,timestamp,cameraid,filename
0,33ae6abe79c54c169264744932ebb551,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:47.397423+00:00,scene-0004,samples/CAM_BACK_LEFT/n015-2018-07-18-11-18-34...
1,33ae6abe79c54c169264744932ebb551,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:47.398588+00:00,scene-0004,samples/LIDAR_TOP/n015-2018-07-18-11-18-34+080...
2,33ae6abe79c54c169264744932ebb551,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:47.404844+00:00,scene-0004,sweeps/CAM_FRONT_LEFT/n015-2018-07-18-11-18-34...
3,33ae6abe79c54c169264744932ebb551,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:47.405673+00:00,scene-0004,samples/RADAR_FRONT_RIGHT/n015-2018-07-18-11-1...
4,33ae6abe79c54c169264744932ebb551,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:47.412466+00:00,scene-0004,sweeps/CAM_FRONT/n015-2018-07-18-11-18-34+0800...
...,...,...,...,...,...
240,f54238bdd48443fcb4a019271dbca2f8,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:50.485202+00:00,scene-0004,sweeps/RADAR_FRONT_LEFT/n015-2018-07-18-11-18-...
241,f54238bdd48443fcb4a019271dbca2f8,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:50.490943+00:00,scene-0004,sweeps/RADAR_BACK_RIGHT/n015-2018-07-18-11-18-...
242,f54238bdd48443fcb4a019271dbca2f8,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:50.500140+00:00,scene-0004,sweeps/LIDAR_TOP/n015-2018-07-18-11-18-34+0800...
243,f54238bdd48443fcb4a019271dbca2f8,ae03d500f9f5482f9fdaf390c796e501,2018-07-17 20:19:50.504844+00:00,scene-0004,sweeps/CAM_FRONT_LEFT/n015-2018-07-18-11-18-34...


In [18]:
end - start

189.28790807724

In [11]:
# for k in id_time_camId_filename:
#     print(k[-2], str(k[-3]), k[-1])
#     print("                                           ", *k[:-3])

In [12]:
resultImages = dict() # maping from image -> (itemId, timestamp, camId) that it was found at
for result in id_time_camId_filename:
    itemId1, itemId2, timestamp, camId, filename = result
    filename = filename.split("/")[-1] # use split so that prefix path is not included in filename
    if filename not in resultImages:
        resultImages[filename] = []
    resultImages[filename].append((itemId1, itemId2, timestamp, camId))

In [13]:
from show_images import show_images
%matplotlib inline

# data_dir =  "data/nuscenes/experiment_data"
data_dir = '/work/apperception/data/nuScenes/full-dataset-v1.0/Trainval/experiment_data'
# data_dir = '/Users/chanwutk/Documents/experiment_data'

In [14]:
show_images(data_dir, resultImages, sample=10)

Sampling 10 out of 245 images.
image n015-2018-07-18-11-18-34+0800__LIDAR_TOP__1531883987398588.pcd.bin
('33ae6abe79c54c169264744932ebb551', 'ae03d500f9f5482f9fdaf390c796e501', datetime.datetime(2018, 7, 17, 20, 19, 47, 398588, tzinfo=datetime.timezone.utc), 'scene-0004')


FileNotFoundError: [Errno 2] No such file or directory: '/work/apperception/data/nuScenes/full-dataset-v1.0/Trainval/experiment_data/n015-2018-07-18-11-18-34+0800__LIDAR_TOP__1531883987398588.pcd.bin'

<Figure size 4320x2160 with 0 Axes>

In [ ]:
expected_imgs = [
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118112404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657119112404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127612404.jpg',
    'n008-2018-09-18-12-07-26-0400__CAM_FRONT__1537287358412404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118612404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127112404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657128112404.jpg',
]
show_images(data_dir, expected_imgs, sample=10)

In [ ]:
missing = 0
missing_images = []
for expected_img in expected_imgs:
    if expected_img not in resultImages.keys():
        missing += 1
        missing_images.append(expected_img)
print("Percentage of expected results missing from query: ", missing, "/", len(expected_imgs), "=", 100 * missing / len(expected_imgs), "%")
show_images(data_dir, missing_images, sample=10)

In [ ]:
extra = 0
extra_images = {}
for img in resultImages.keys():
    if img not in expected_imgs:
        extra += 1
        extra_images[img] = resultImages[img]
print("Percentage of images in query but not in expected results: ", extra, "/", len(resultImages.keys()), "=", 100 * extra / len(resultImages.keys()), "%")
show_images(data_dir, extra_images, sample=10, seed=5)